In [ ]:
import numpy as np
import rclpy
from rclpy.node import Node
from cv_bridge import CvBridge
from vision.owl import owl_vit
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from vision.util import get_det_points, Rotation_X, Rotation_Y, Translation, HT_matrix
import math
import cv2
from vision.detector import compute_xyz

: 

In [67]:
class ImageListener(Node):
    def __init__(self, queries, person=False):
        super().__init__('image_listener')
        self.model = owl_vit()
        self.object_queries = queries
        self.model.query_text(queries)
        print("finish loading model")
        self.cv_bridge = CvBridge()
        info_sub = message_filters.Subscriber(self, CameraInfo, '/camera/color/camera_info')
        rbg_sub = message_filters.Subscriber(self, Image, '/camera/color/image_raw')
        depth_sub = message_filters.Subscriber(self, Image, '/camera/aligned_depth_to_color/image_raw')
        ts = message_filters.ApproximateTimeSynchronizer([info_sub, rbg_sub, depth_sub], 10, 1, allow_headerless=True)
        ts.registerCallback(self.callback)
        self.depth_cv = None
        self.rgb_cv = None
        self.det_vis_pub = self.create_publisher(Image, '/det_vis', 10)
        if person:
            self.human_sub = self.create_subscription(Image, '/zed/image_raw', self.human_callback, 10)
            self.human_det_pub = self.create_publisher(Image, '/human_det_vis', 10)
    def human_callback(self, msg):
        self.human_cv = self.cv_bridge.imgmsg_to_cv2(msg)
        print("got human image")

    def callback(self, info_msg, rgb_msg, depth_msg):
        # camera info
        print(info_msg)
        intrinsics = np.array(info_msg.p).reshape(3, 4)
        self.fx = intrinsics[0, 0]
        self.fy = intrinsics[1, 1]
        self.px = intrinsics[0, 2]
        self.py = intrinsics[1, 2]
        self.intrinsics = intrinsics
        print("got image")
        if depth_msg.encoding == '32FC1':
            depth_cv = self.cv_bridge.imgmsg_to_cv2(depth_msg)
        elif depth_msg.encoding == '16UC1':
            depth_cv = self.cv_bridge.imgmsg_to_cv2(depth_msg).copy().astype(np.float32)
            depth_cv /= 1000.0
        else:
            self.get_logger().error('Depth image has unsupported encoding: %s' % depth_msg.encoding)
            return
        rbg_cv = self.cv_bridge.imgmsg_to_cv2(rgb_msg)
        self.depth_cv = depth_cv
        self.rgb_cv = rbg_cv

    def detect_human(self, human_queries, save_indx = None):
        self.model.query_text(human_queries)
        if self.human_cv is None:
            print("no human image")
            return None
        print("start detecting")
        if save_indx is not None:
            cv2.imwrite("./data/det/human_ori_{}.jpg".format(save_indx), self.human_cv)
        # inp = cv2.cvtColor(self.human_cv, cv2.COLOR_BGR2RGB)
        im_color,_, boxes, found_object_names = self.model.detect(self.human_cv, thres=0.1)
        if save_indx is not None:
            cv2.imwrite("./data/det/human_det_{}.jpg".format(save_indx), im_color)
        self.human_det_pub.publish(self.cv_bridge.cv2_to_imgmsg(im_color))
        return boxes, found_object_names
    
    def detect_object(self, save_indx = None):
        self.model.query_text(self.object_queries)
        if self.depth_cv is None or self.rgb_cv is None:
            print("no image")
            return None, None
        print("start detecting")
        inp = self.rgb_cv #cv2.cvtColor(self.rgb_cv, cv2.COLOR_BGR2RGB)
        if save_indx is not None:
            cv2.imwrite("./data/det/ori_{}.jpg".format(save_indx), inp)
        xyz_img = compute_xyz(self.depth_cv, self.fx, self.fy, self.px, self.py, self.depth_cv.shape[0], self.depth_cv.shape[1])
        im_color,index_mask, boxes, found_object_names = self.model.detect(inp, thres=0.05)
        self.det_vis_pub.publish(self.cv_bridge.cv2_to_imgmsg(im_color))
        objects_xyz = self.get_position(index_mask, xyz_img)
        if save_indx is not None:
            im_color = cv2.cvtColor(im_color, cv2.COLOR_BGR2RGB)
            cv2.imwrite("./data/det/det_{}.jpg".format(save_indx), im_color)
        return objects_xyz, found_object_names
    
    def get_position(self, label_mask, xyz_img):
        # Calibration
        rotation_x   = Rotation_X(-math.pi)
        rotation_y   = Rotation_Y(-math.pi/4)
        rotation_mat = np.dot(rotation_x, rotation_y)
        position_mat = Translation(0.454,0.01,1.33)
        transform_mat= HT_matrix(rotation_mat, position_mat)
        objects_xyz = get_det_points(depth_pixel=xyz_img, label_pixel=label_mask, transform_mat=transform_mat)
        return objects_xyz

In [68]:
rclpy.init()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_28231/1494223770.py", line 1, in <module>
    rclpy.init()
  File "/opt/ros/humble/local/lib/python3.10/dist-packages/rclpy/__init__.py", line 89, in init
    return context.init(args, domain_id=domain_id)
  File "/opt/ros/humble/local/lib/python3.10/dist-packages/rclpy/context.py", line 70, in init
    raise RuntimeError('Context.init() must only be called once')
RuntimeError: Context.init() must only be called once

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/ultratb.p

In [5]:
node.destroy_node()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_28231/4098198773.py", line 1, in <module>
    node.destroy_node()
NameError: name 'node' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1319, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/ultr

In [69]:
node = ImageListener(['coca cola can', 'fanta can','monster can', 'coffee can', 'lemon', 'redbull can',
                      'apple', 'starbucks can', 'orange', 'pepsi can'], person=True)

start loading owl-vit
End Loading
finish loading model


In [211]:
while node.depth_cv is None or node.rgb_cv is None:
    rclpy.spin_once(node, timeout_sec=1.0)
rclpy.spin_once(node, timeout_sec=1.0)

In [206]:
node.object_queries = ['coke can', 'fanta can', 'starbucks can', 'lemon', 'redbull can',
                      'apple', 'redbull can', 'orange', 'monster can']#, 'pepsi can']

In [212]:
objects_xyz, found_object_names = node.detect_object(save_indx=5)

start detecting
tensor([0.3448, 0.1857, 0.3824, 0.0601, 0.0826, 0.7350])
Detected a photo of a apple with confidence 0.735 at location [983.63, 355.46, 1099.34, 456.41]
Detected a photo of a coke can with confidence 0.382 at location [411.9, 273.54, 499.74, 425.67]
Detected a photo of a monster can with confidence 0.345 at location [572.19, 168.8, 644.27, 342.73]
Detected a photo of a monster can with confidence 0.186 at location [802.94, 216.22, 877.44, 355.39]
[0 1 2 3 4]
Object Center [0.8492119454843259, -0.3183995691242988, 0.82]
Object Center [0.8679250407838031, 0.15704504807895994, 0.82]
Object Center [0.9670021580309515, 0.03566637844222247, 0.82]
Object Center [0.9315438274324427, -0.15954880718569775, 0.82]


In [216]:
while node.depth_cv is None or node.rgb_cv is None:
    rclpy.spin_once(node, timeout_sec=1.0)
rclpy.spin_once(node, timeout_sec=1.0)

got human image


In [217]:
human_boxes, human_names = node.detect_human(['person wearing gray shirt', 
                                              'person wearing green shirt', 
                                              'person wearing black shirt', 'person wearing blue shirt']
                                              , save_indx=5)

start detecting
tensor([0.1867, 0.2380, 0.1813, 0.1121, 0.2470])
Detected a photo of a person wearing black shirt with confidence 0.247 at location [82.8, 243.92, 378.98, 589.53]
Detected a photo of a person wearing black shirt with confidence 0.238 at location [656.64, 145.58, 752.01, 441.05]
Detected a photo of a person wearing black shirt with confidence 0.187 at location [517.9, 151.09, 624.52, 398.56]
Detected a photo of a person wearing green shirt with confidence 0.181 at location [412.7, 167.76, 518.4, 449.48]


In [218]:
centers = [(box[0] + box[2])/2 for box in human_boxes]
centers = np.array(centers)
sorted_index = np.argsort(centers)



In [219]:
human_data = []
for i in range(len(centers)):
    indx = int(3 - sorted_index[i])
    human_data.append(indx)

In [220]:
human_data, human_names

([3, 0, 1, 2],
 ['a photo of a person wearing black shirt',
  'a photo of a person wearing black shirt',
  'a photo of a person wearing black shirt',
  'a photo of a person wearing green shirt'])

In [81]:
rclpy.shutdown()

In [30]:
objects_xyz, found_object_names

([[1.0119115163426762, 0.04988672025961422, 0.82],
  [0.9535481805901102, -0.237059144786264, 0.82],
  [0.9281271281774758, -0.0837396599989099, 0.82],
  [0.9109748339821183, 0.230179104231586, 0.82]],
 ['a photo of a apple',
  'a photo of a pepsi can',
  'a photo of a pepsi can',
  'a photo of a monster can'])

In [221]:
object_names = [name.replace("a photo of a","") for name in found_object_names]
human_names = [name.replace("a photo of a","") for name in human_names]

## Save

In [222]:
det_res = {
        "objects_xyz": objects_xyz, "object_name": object_names,
        'person_name': human_names, 'person_data': human_data
    }

In [223]:
det_res

{'objects_xyz': [[0.8492119454843259, -0.3183995691242988, 0.82],
  [0.8679250407838031, 0.15704504807895994, 0.82],
  [0.9670021580309515, 0.03566637844222247, 0.82],
  [0.9315438274324427, -0.15954880718569775, 0.82]],
 'object_name': [' apple', ' coke can', ' monster can', ' monster can'],
 'person_name': [' person wearing black shirt',
  ' person wearing black shirt',
  ' person wearing black shirt',
  ' person wearing green shirt'],
 'person_data': [3, 0, 1, 2]}

In [151]:
import json
class parser():
    def __init__(self,cat='certain', idx=0, trial=0):
        self.cat = cat
        self.index = idx
        self.trial = trial

In [224]:
args = parser('demo', 5)
with open("./data/det/{}_{}.json".format(args.cat, args.index), "w") as f:
    json.dump(det_res, f, indent=4)

### Parse

In [ ]:
with open("./data/det/{}_{}.json".format(args.cat, args.index), "r") as f:
    data = json.load(f)
objects_xyz = data['objects_xyz']

## Move robot

In [ ]:
rclpy.init()

: 

In [ ]:
from model.robot_client import RobotClient

2023-10-04 15:33:14,604	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [37]:
client = RobotClient()

[INFO] [1696401194.815115334] [minimal_client_async]: Successfully connected to action '/scaled_joint_trajectory_controller/follow_joint_trajectory'


window
Initial Joint values: [-1.57079633 -2.31186313  2.14413699  1.7392206   0.78539816 -1.57114539]
Solved IK: [-5.03303745e-01 -1.04713223e+00  2.53353154e+00 -1.48565632e+00
  1.06600166e+00 -3.99537744e-04]
Done.


In [230]:
client.reset_capture_pose()

[(builtin_interfaces.msg.Duration(sec=7, nanosec=0), array([-1.57079633, -2.31186313,  2.14413699,  1.7392206 ,  0.78539816,
       -1.57114539]))]


[INFO] [1696404566.908883297] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'


In [153]:
rclpy.spin_once(client, timeout_sec=1.0)

In [155]:
print(client.current_q)

[-1.57080967 -2.31184735  2.14411146  1.73921601  0.78538108 -1.57113344]


In [231]:
pick_pos = np.asarray(objects_xyz[0])
client.pick(pick_pos)
client.give(1)

Initial Joint values: [-5.03303745e-01 -1.04713223e+00  2.53353154e+00 -1.48565632e+00
  1.06600166e+00 -3.99537744e-04]
Solved IK: [-0.58722189 -0.80615712  1.85751721 -1.05310583  0.9838852 ]
Done.


[INFO] [1696404588.946433203] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'
[INFO] [1696404600.650213100] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'


[ 1.00091379 -0.159369    0.81978394] [-5.17500226e-01 -7.15624408e-01  1.63801557e+00 -9.24048619e-01
  1.05312073e+00 -4.22779714e-04]
Initial Joint values: [-5.17500226e-01 -7.15624408e-01  1.63801557e+00 -9.24048619e-01
  1.05312073e+00 -4.22779714e-04]
Solved IK: [-0.51749989 -1.00536923  1.57818277 -0.57444329  1.05344204]
[-5.17499886e-01 -1.00536923e+00  1.57818277e+00 -5.74443289e-01
  1.05344204e+00 -4.22779714e-04]
Done.


[INFO] [1696404605.092978498] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'


In [56]:
client.give(2)

[ 1.02138185 -0.23695304  0.81979554] [-6.11429040e-01 -6.46072463e-01  1.47676164e+00 -8.32377688e-01
  9.59498048e-01 -4.10381948e-04]
Initial Joint values: [-6.11429040e-01 -6.46072463e-01  1.47676164e+00 -8.32377688e-01
  9.59498048e-01 -4.10381948e-04]
Solved IK: [-0.61151962 -0.91193703  1.41696613 -0.50680236  0.95954654]
[-6.11519619e-01 -9.11937032e-01  1.41696613e+00 -5.06802365e-01
  9.59546539e-01 -4.10381948e-04]
Done.


[INFO] [1696402174.533656370] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'


In [232]:
client.reset_from_back()

[INFO] [1696404630.187022588] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'


In [200]:
client.infeasible_callback()

[INFO] [1696404057.065617887] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'
[INFO] [1696404065.270731215] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'


In [13]:
import time
deg = 30

q_array = np.vstack([client.current_q, client.pre_pose])
client.send_trajectory(q_array, vel=30)
time.sleep(2)
capture_pose_q = client.capture_q.copy()
rad = deg/180.0*np.pi
left = capture_pose_q.copy()
left[0] += rad
right = capture_pose_q.copy()
right[0] -= rad
q_array = np.vstack([client.pre_pose, capture_pose_q, left, right])
for _ in range(1):
    q_array = np.vstack([q_array, left, right])
q_array = np.vstack([q_array, capture_pose_q])
client.send_trajectory(q_array, vel=25)

[INFO] [1696355424.195383480] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'
[INFO] [1696355426.249750423] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'
